# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [1]:
!git clone https://github.com/KU-HIAI/tomok.git

Cloning into 'tomok'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 262 (delta 104), reused 150 (delta 61), pack-reused 58
Receiving objects: 100% (262/262), 15.78 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (112/112), done.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [2]:
cd tomok

/content/tomok


## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [4]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS241421_04010103_03)
class KDS241421_04010103_03 (tomok.RuleUnit):

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Chanwoo Yang'  # 작성자명
    ref_code = 'KDS 24 14 21 4.1.1.3 (3)' # 건설기준문서
    ref_date = '2021-04-12'  # 디지털 건설문서 작성일
    doc_date = '2023-11-14'  # 건설기준문서->디지털 건설기준 변환 기준일
    title = '축력과 2축 휨이 작용하는 부재의 휨강도'    # 건설기준명

    #
    description = """
    콘크리트교 설계기준 (한계상태설계법)
    4. 설계
    4.1 극한한계상태
    4.1.1 휨과 축력
    4.1.1.3 축력과 2축 휨이 작용하는 부재의 휨강도
    (3)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """

    """
    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
    subgraph Python_Class
    A["축력과 2축 휨이 작용하는 부재의 휨강도"];
    B["KDS 24 14 21 4.1.1.3 (3)"];
    A ~~~ B
    end

		subgraph Variable_def;
		VarIn1[/입력변수: 2차 모멘트를 포함하는 y축에 대한 계수휨모멘트/];
		VarIn2[/입력변수: 2차 모멘트를 포함하는 z축에 대한 계수휨모멘트/];
    VarIn3[/입력변수: y축에 대한 설계휨강도/] ;
		VarIn4[/입력변수: z축에 대한 설계휨강도/] ;
		VarIn5[/입력변수: 단면 형상과 축력비에 따른 지수/];
		VarIn6[/입력변수: 계수하중에 의한 축력/];
		VarIn7[/입력변수: 단면의 설계중심축압축강도/];

		VarIn1 & VarIn2 & VarIn3 & VarIn4
		VarIn2 & VarIn3 & VarIn4~~~VarIn5 & VarIn6 & VarIn7
		end
		Python_Class ~~~ Variable_def;
		Variable_def--->D
		D{"단면 형상과 축력비에 따른 지수"}
		D--직사각형 단면인 경우-->

		C{"<img src='https://latex.codecogs.com/svg.image?N_{u}/N_{od}'>---------------------------------"}

		C--0.7-->E[α=1.5]
		C--1.0-->F[α=2.0]
		C--0.1보다 작거나 같을때-->G[α=1.0]
		D--원형단면과 타원형 단면인 경우--> H[α=2.0]
		E & F & G & H--->I--->K
		I{"<img src='https://latex.codecogs.com/svg.image?(\frac{M_{uy}}{M_{dy}})^{\alpha}&plus;(\frac{M_{uz}}{M_{dz}})^{\alpha}\leq&space;1.0'>---------------------------------"}
		K(["Pass or Fail"])
    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정,
    @tomok.rule_method
    def bending_strength_of_member_subject_to_axial_force_and_biaxial_bending(fIMuy,fIMuz,fIMdy,fIMdz,fIalpha,fINu,fINod,fIuserdefined) -> bool:
        """축력과 2축 휨이 작용하는 부재의 휨강도

        Args:
             fIMuy (float): 2차 모멘트를 포함하는 y축에 대한 계수휨모멘트
             fIMuz (float): 2차 모멘트를 포함하는 z축에 대한 계수휨모멘트
             fIMdy (float): y축에 대한 설계휨강도
             fIMdz (float): z축에 대한 설계휨강도
             fIalpha (float): 단면 형상과 축력비에 따른 지수
             fINu (float): 계수하중에 의한 축력
             fINod (float): 단면의 설계중심축압축강도
             fIuserdefined (float): 사용자 선택


        Returns:
            bool: 콘크리트교 설계기준 (한계상태설계법) 4.1.1.3 축력과 2축 휨이 작용하는 부재의 휨강도 (3)의 통과 여부
        """

        #원형 단면과 타원형 단면의 경우: fIuserdefined = 1
        #직사각형 단면의 경우: fIuserdefined = 2
        if fIuserdefined == 1:
          fIalpha = 2.0
        elif fIuserdefined == 2:
          fIvariable = fINu / fINod
          if fIvariable <= 0.1:
            fIalpha = 1.0
          elif fIvariable > 0.1 and fIvariable < 0.7:
            fIalpha = (0.5 / 0.6) * (fIvariable - 0.7) + 1.5
          elif fIvariable == 0.7:
            fIalpha = 1.5
          elif fIvariable > 0.7 and fIvariable < 1.0:
            fIalpha = (0.5 / 0.3) * (fIvariable - 0.7) + 1.5
          elif fIvariable == 1.0:
            fIalpha = 2.0
        if (fIMuy/fIMdy)**fIalpha + (fIMuz/fIMdz)**fIalpha <= 1.0:
          return "Pass"
        else:
          return "Fail"

No stream support: No module named 'lark'


작성한 룰 유닛은 아래의 코드 블럭과 같이 생성하여, 작성자가 임의로 검증을 수행할 수 있습니다.

In [5]:
my_RuleUnit = KDS241421_04010103_03()

In [6]:
fIMuy = 200
fIMuz = 200
fIMdy = 500
fIMdz = 400
fIalpha = None
fINu = 70
fINod = 100
fIuserdefined = 2

In [7]:
Rule_Review_Result = my_RuleUnit.bending_strength_of_member_subject_to_axial_force_and_biaxial_bending(fIMuy,fIMuz,fIMdy,fIMdz,fIalpha,fINu,fINod,fIuserdefined)
# 해당건설기준 항목 를 통과하는가?

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: Pass


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [8]:
my_RuleUnit.render_markdown()